<a href="https://colab.research.google.com/github/apdubeyko1/TB-1/blob/main/Telegram_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyTelegramBotAPI

In [ ]:
pip install transformers==4.24.0

In [ ]:
pip install xlrd

In [ ]:
pip install google-drive-to-sqlite auth

In [ ]:

import telebot
from telebot import types
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import torch
import xlrd
import sqlite3

TOKEN = "5958589091:AAH26SHZd4EACGRdFCIEbDIyK2LdkCqqOTo"
bot = telebot.TeleBot(TOKEN)
np.random.seed(42)
torch.manual_seed(42)
path = "/content/drive/My Drive/BOT/databases/mydatabase.db"
conn = sqlite3.connect(path)
cursor = conn.cursor()

def load_tokenizer_and_model(model_name_or_path):
	return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

def db_table_val(user_id: int, user_name: str, username: str):
	cursor.execute('INSERT INTO test (user_id, user_name, username) VALUES (?, ?, ?)', (user_id, user_name, username))
	conn.commit()

def generate(
		model, tok, text,
		do_sample=True, max_length=50, repetition_penalty=5.0,
		top_k=5, top_p=0.95, temperature=1,
		num_beams=None,
		no_repeat_ngram_size=3
):
	input_ids = tok.encode(text, return_tensors="pt").cuda()
	out = model.generate(
		input_ids.cuda(),
		max_length=max_length,
		repetition_penalty=repetition_penalty,
		do_sample=do_sample,
		top_k=top_k, top_p=top_p, temperature=temperature,
		num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
	)
	return list(map(tok.decode, out))


tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3medium_based_on_gpt2")

print('MODEL_GPT SUCCESSFUL')


@bot.message_handler(commands=['start'])
def send_welcome(message):
    stic = open('/content/drive/MyDrive/BOT/stic.webp', 'rb')
    bot.reply_to(message, "Привет, {0.first_name}\n/Меня зовут Алекс.".format(message.from_user),parse_mode='html')
    bot.send_sticker(message.chat.id,stic)
    bot.send_message(message.chat.id, 'Используй команды:\n/Привет - для общения\n и /help для ознакомления')
    bot.send_message(message.chat.id, 'ОБЯЗАТЕЛЬНО СПЕРВА ПОСМОТРИ ИНФОРМАЦИЮ (/help)')
    stic = open('/content/drive/MyDrive/BOT/stic.webp', 'rb')
	 
	# клавиатура
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    but1 = types.KeyboardButton("Наше пиво")
    but2 = types.KeyboardButton("Наши снэки")
    markup.add(but1, but2)
    bot.reply_to(message, "Привет, {0.first_name}\n Я рад предложить вам нашу продукцию".format(message.from_user),parse_mode='html',reply_markup=markup)
	

@bot.message_handler(func=lambda message: True)
def menu(message):
    if message.chat.type == 'private':
        if message.text == "Наше пиво":
            rb = xlrd.open_workbook('/content/drive/MyDrive/BOT/price.xls', formatting_info=True)
            sheet = rb.sheet_by_index(0)
            vals = [sheet.row_values(rownum) for rownum in range(sheet.nrows)]
            bot.send_message(message.chat.id, vals)
            
        elif message.text == "Наши снэки":
            rb = xlrd.open_workbook('/content/drive/MyDrive/BOT/снэки.xls', formatting_info=True)
            sheet = rb.sheet_by_index(0)
            vals = [sheet.row_values(rownum) for rownum in range(sheet.nrows)]
            bot.send_message(message.chat.id, vals)

        elif message.text.lower() == 'да':
            bot.send_message(message.from_user.id, 'Привет! Ваше имя добавленно в базу данных!')
    
            us_id = message.from_user.id
            us_name = message.from_user.first_name
            username = message.from_user.username
		
            db_table_val(user_id=us_id, user_name=us_name, username=username)
        else:
            bot.send_message(message.chat.id, "Пожалуйста, напишите команду /start или /help")


@bot.message_handler(commands=['привет'], content_types=['text'])
def send_message(message):
	try:
		user = message.text
		user = user.split('!')
		user_length = int(user[1])
		user_text = user[2]
		generator = generate(model, tok, text=user_text, num_beams=1, max_length=user_length)
		# print(f'user: {message.from_user.first_name} \nmessage:\n {user}\ngenerated text:\n{generator[0]}\n') # IT'S FOR ME
		# print(user)
		# bot.send_message(message.chat.id, f'слов:{next_words}')
		# bot.send_message(message.chat.id, f'seed: {seed_text}')
		bot.send_message(message.chat.id, text=generator[0])
	except:
		bot.send_message(message.chat.id, 'неправильный ввод! смотри /help')


@bot.message_handler(commands=['help'])
def send_info(message):
	bot.send_message(message.chat.id, 'Тебе необходимо передать мне две вещи\n'
									  'самой первой идет команда /привет\n'
									  'затем восклицательный знак и твое обращение\n'
									  'восклицательный знак я использую как разделитель\n'
									  'я покажу пример, как это должно выглядеть:\n'
									  '/Привет !я пошел в лес и встретил\n'
									  'после чего, я смогу ответить\n')


bot.polling()